In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import string
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.stem.porter import PorterStemmer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, SimpleRNN
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

In [6]:
basketball = pd.read_csv("/content/drive/My Drive/Tolu/tolufilebasketball.csv")


tennis = pd.read_csv("/content/drive/My Drive/Tolu/tolufiletennis.csv")

cricket = pd.read_csv("/content/drive/My Drive/Tolu/tolufilecricket.csv")

In [7]:
headlines = pd.concat([basketball, tennis, cricket], axis=0)

In [8]:
headlines = headlines.sample(frac = 1)

In [9]:
headlines

,title,author,published_date,published_date_precision,link,clean_url,excerpt,summary,rights,rank,topic,country,language,authors,media,is_opinion,twitter_account,_score,_id
675,Duncanville stripped of 2021-22 UIL 6A boys ba...,Andrew McCulloch,2022-10-31 20:00:37,full,https://news.scorebooklive.com/texas/2022/10/3...,scorebooklive.com,The Panthers were stripped of their most recen...,The Duncanville Panthers have been stripped of...,scorebooklive.com,38179,news,US,en,Andrew McCulloch,https://news.scorebooklive.com/.image/t_share/...,False,NaN,10.637465,48bff7f6ab1c651bfe283b1ee34bb1a8
2226,Litigant Alleges Georgia Judge Has 'Close Pers...,Cedra Mayfield,2022-10-31 14:24:00,full,https://www.law.com/dailyreportonline/2022/10/...,law.com,Maintaining an employment relationship with a ...,A Georgia college basketball coach is seeking ...,law.com,1936,news,US,en,"Cedra Mayfield,The Associated Press",https://images.law.com/contrib/content/uploads...,False,@DailyReport,10.209267,e42ae76ef3a424ad2236e074fffe37c4
3447,"Gobert out for Wolves, placed in health and sa...",Brian Hall,2022-11-05 23:28:50,full,https://www.nwahomepage.com/news/national-spor...,nwahomepage.com,MINNEAPOLIS (AP) — Rudy Gobert will miss the M...,Minnesota Timberwolves center Rudy Gobert (27)...,nwahomepage.com,19086,news,US,en,"Associated Press,BRIAN HALL",https://www.nwahomepage.com/wp-content/uploads...,False,NaN,10.012569,36ed3dd0ca3383a9eab70db58353de1e
258,On the Go with Ayo at Padelhorn,Ayo Elise,2022-11-03 16:59:00,full,https://www.kptv.com/2022/11/03/go-with-ayo-pa...,kptv.com,A sport that is gaining popularity overseas is...,"VANCOUVER, Wash. (KPTV) - A sport that is gain...",kptv.com,9177,news,US,af,Ayo Elise,https://gray-kptv-prod.cdn.arcpublishing.com/r...,False,@fox12oregon,11.863000,040c84d722221527a7069fceb2706d8c
4664,Louisville escapes major sanctions in hoops br...,Gary B. Graves,2022-11-03 15:25:28,full,https://www.nwahomepage.com/news/national-spor...,nwahomepage.com,"LOUISVILLE, Ky. (AP) — After five agonizing ye...","LOUISVILLE, Ky. (AP) — After five agonizing ye...",nwahomepage.com,19086,news,US,en,"Associated Press,GARY B. GRAVES",https://www.nwahomepage.com/wp-content/uploads...,False,NaN,9.844439,a8a31d31a471d5dcc2e2e977267b6484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3958,"Mitchell, Love rally Cavaliers past Knicks, 12...",Brian Dulik,2022-10-31 00:42:48,full,https://www.timesunion.com/sports/article/Mitc...,timesunion.com,CLEVELAND (AP) — Donovan Mitchell had 38 point...,This is a carousel. Use Next and Previous butt...,timesunion.com,2528,sport,US,en,"Associated Press,BRIAN DULIK",https://s.hdnux.com/photos/01/30/11/06/2310598...,False,@TimesUnion,9.941422,fcc7945e2dfc19374895deebcb0d0bc3
703,InventHelp Inventor Develops New Training Aid ...,PRNewswire,2022-11-01 00:00:00,date,https://www.mysuncoast.com/prnewswire/2022/11/...,mysuncoast.com,"PITTSBURGH, Nov. 1, 2022 /PRNewswire/ -- ""I wa...","PITTSBURGH, Nov. 1, 2022 /PRNewswire/ -- ""I wa...",mysuncoast.com,24998,news,US,en,PRNewswire,https://www.mysuncoast.com/pf/resources/images...,False,@mysuncoast,11.316863,657a134a6836f2a9a597192f7ec8f751
2185,NOW OPEN: LUXURY WELLNESS RETREAT SENSEI PORCU...,PRNewswire,2022-11-01 00:00:00,date,https://www.kold.com/prnewswire/2022/11/01/now...,kold.com,"LOS ANGELES, Nov. 1, 2022 /PRNewswire/ -- Toda...","LOS ANGELES, Nov. 1, 2022 /PRNewswire/ -- Toda...",kold.com,15186,news,US,en,PRNewswire,https://www.kold.com/pf/resources/images/logo-...,False,@KOLDNews,9.458585,0a0d281347bff44ee9ba6396edd4d020
2064,Hartford men's basketball coach John Gallagher...,NaN,2022-11-08 19:22:04,full,https://www.seattletimes.com/sports/hartford-m...,seattletimes.com,NaN,Coach John Gallagher has resigned less than tw...,NaN,810,news,US,en,NaN,NaN,False,NaN,10.242921,0f3e83e260ad8b396bd55342ca66f494


In [10]:
def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", str(text))


def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)
def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)
def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)
  

In [11]:
headlines["title"] = headlines.title.map(lambda x: remove_URL(x))
headlines["title"] = headlines.title.map(lambda x: remove_html(x))
headlines["title"] = headlines.title.map(lambda x: remove_emoji(x))
headlines["title"] = headlines.title.map(lambda x: remove_punct(x))

In [12]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(text)
headlines["title"] =headlines["title"].map(remove_stopwords)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
headlines

,title,author,published_date,published_date_precision,link,clean_url,excerpt,summary,rights,rank,topic,country,language,authors,media,is_opinion,twitter_account,_score,_id
675,duncanville stripped 202122 uil 6a boys basket...,Andrew McCulloch,2022-10-31 20:00:37,full,https://news.scorebooklive.com/texas/2022/10/3...,scorebooklive.com,The Panthers were stripped of their most recen...,The Duncanville Panthers have been stripped of...,scorebooklive.com,38179,news,US,en,Andrew McCulloch,https://news.scorebooklive.com/.image/t_share/...,False,NaN,10.637465,48bff7f6ab1c651bfe283b1ee34bb1a8
2226,litigant alleges georgia judge close personal ...,Cedra Mayfield,2022-10-31 14:24:00,full,https://www.law.com/dailyreportonline/2022/10/...,law.com,Maintaining an employment relationship with a ...,A Georgia college basketball coach is seeking ...,law.com,1936,news,US,en,"Cedra Mayfield,The Associated Press",https://images.law.com/contrib/content/uploads...,False,@DailyReport,10.209267,e42ae76ef3a424ad2236e074fffe37c4
3447,gobert wolves placed health safety protocols,Brian Hall,2022-11-05 23:28:50,full,https://www.nwahomepage.com/news/national-spor...,nwahomepage.com,MINNEAPOLIS (AP) — Rudy Gobert will miss the M...,Minnesota Timberwolves center Rudy Gobert (27)...,nwahomepage.com,19086,news,US,en,"Associated Press,BRIAN HALL",https://www.nwahomepage.com/wp-content/uploads...,False,NaN,10.012569,36ed3dd0ca3383a9eab70db58353de1e
258,go ayo padelhorn,Ayo Elise,2022-11-03 16:59:00,full,https://www.kptv.com/2022/11/03/go-with-ayo-pa...,kptv.com,A sport that is gaining popularity overseas is...,"VANCOUVER, Wash. (KPTV) - A sport that is gain...",kptv.com,9177,news,US,af,Ayo Elise,https://gray-kptv-prod.cdn.arcpublishing.com/r...,False,@fox12oregon,11.863000,040c84d722221527a7069fceb2706d8c
4664,louisville escapes major sanctions hoops bribe...,Gary B. Graves,2022-11-03 15:25:28,full,https://www.nwahomepage.com/news/national-spor...,nwahomepage.com,"LOUISVILLE, Ky. (AP) — After five agonizing ye...","LOUISVILLE, Ky. (AP) — After five agonizing ye...",nwahomepage.com,19086,news,US,en,"Associated Press,GARY B. GRAVES",https://www.nwahomepage.com/wp-content/uploads...,False,NaN,9.844439,a8a31d31a471d5dcc2e2e977267b6484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3958,mitchell love rally cavaliers past knicks 121108,Brian Dulik,2022-10-31 00:42:48,full,https://www.timesunion.com/sports/article/Mitc...,timesunion.com,CLEVELAND (AP) — Donovan Mitchell had 38 point...,This is a carousel. Use Next and Previous butt...,timesunion.com,2528,sport,US,en,"Associated Press,BRIAN DULIK",https://s.hdnux.com/photos/01/30/11/06/2310598...,False,@TimesUnion,9.941422,fcc7945e2dfc19374895deebcb0d0bc3
703,inventhelp inventor develops new training aid ...,PRNewswire,2022-11-01 00:00:00,date,https://www.mysuncoast.com/prnewswire/2022/11/...,mysuncoast.com,"PITTSBURGH, Nov. 1, 2022 /PRNewswire/ -- ""I wa...","PITTSBURGH, Nov. 1, 2022 /PRNewswire/ -- ""I wa...",mysuncoast.com,24998,news,US,en,PRNewswire,https://www.mysuncoast.com/pf/resources/images...,False,@mysuncoast,11.316863,657a134a6836f2a9a597192f7ec8f751
2185,open luxury wellness retreat sensei porcupine ...,PRNewswire,2022-11-01 00:00:00,date,https://www.kold.com/prnewswire/2022/11/01/now...,kold.com,"LOS ANGELES, Nov. 1, 2022 /PRNewswire/ -- Toda...","LOS ANGELES, Nov. 1, 2022 /PRNewswire/ -- Toda...",kold.com,15186,news,US,en,PRNewswire,https://www.kold.com/pf/resources/images/logo-...,False,@KOLDNews,9.458585,0a0d281347bff44ee9ba6396edd4d020
2064,hartford mens basketball coach john gallagher ...,NaN,2022-11-08 19:22:04,full,https://www.seattletimes.com/sports/hartford-m...,seattletimes.com,NaN,Coach John Gallagher has resigned less than tw...,NaN,810,news,US,en,NaN,NaN,False,NaN,10.242921,0f3e83e260ad8b396bd55342ca66f494


In [14]:
vocabulary_size = 10000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(headlines['title'])
sequences = tokenizer.texts_to_sequences(headlines['title'])
X_RNN = pad_sequences(sequences, maxlen=50)

In [16]:
## Split the data into train and test
Y_RNN = headlines["title"]
X_train_RNN, X_test_RNN, Y_train_RNN, Y_test_RNN = train_test_split(X_RNN, 
                       Y_RNN, test_size=0.3, random_state=7)

In [ ]:
X_test_RNN.shape

(3000, 50)

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
def create_model(input_length=50):
  model = Sequential()
  model.add(Embedding(10000, 300, input_length=50))
  model.add(SimpleRNN(24, return_sequences=False))
  model.add(Dense(3, activation='sigmoid'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  
  return model    
model_RNN = KerasClassifier(build_fn=create_model, epochs=3, verbose=1, validation_split=0.4)
model_RNN.fit(X_train_RNN, Y_train_RNN)

<ipython-input-18-3de03d7ed100>:9: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_RNN = KerasClassifier(build_fn=create_model, epochs=3, verbose=1, validation_split=0.4)


Epoch 1/3
132/132 [==============================] - 8s 53ms/step - loss: 0.5075 - accuracy: 0.7971 - val_loss: 0.2950 - val_accuracy: 0.8739
Epoch 2/3
132/132 [==============================] - 7s 52ms/step - loss: 0.0898 - accuracy: 0.9824 - val_loss: 0.3059 - val_accuracy: 0.8732
Epoch 3/3
132/132 [==============================] - 7s 50ms/step - loss: 0.0288 - accuracy: 0.9933 - val_loss: 0.2864 - val_accuracy: 0.8832


In [ ]:
# performance metrics
import sklearn
from sklearn.metrics import classification_report
y_pred = model_RNN.predict(X_test_RNN)
report = classification_report(Y_test_RNN, y_pred.round())
print(report)

94/94 [==============================] - 1s 6ms/step
              precision    recall  f1-score   support

           0       0.78      0.81      0.79       733
           1       0.95      0.98      0.96      1536
           2       0.87      0.79      0.83       731

    accuracy                           0.89      3000
   macro avg       0.87      0.86      0.86      3000
weighted avg       0.89      0.89      0.89      3000

